In [1]:
""" Python main script of MatMat trade module

    Notes
    ------
    Fill notes if necessary

    """

' Python main script of MatMat trade module\n\n    Notes\n    ------\n    Fill notes if necessary\n\n    '

# Imports

In [2]:
# general
import sys
import os
import copy

# scientific
import numpy as np
import pandas as pd
import pymrio
import matplotlib.pyplot as plt

# local folder
from local_paths import data_dir
from local_paths import output_dir

# local library
from utils import Tools

# SETTINGS

In [3]:
# year to study in [*range(1995, 2022 + 1)]
base_year = 2015

# system type: pxp or ixi
system = 'pxp'

# agg name: to implement in agg_matrix.xlsx
agg_name = {
	'sector': 'ref',
	'region': 'ref'
}

# define filename concatenating settings
concat_settings = str(base_year) + '_' + \
	agg_name['sector']  + '_' +  \
	agg_name['region']

# set if rebuilding calibration from exiobase
calib = False

# READ/ORGANIZE/CLEAN DATA

In [4]:
# define file name
file_name = 'IOT_' + str(base_year) + '_' + system + '.zip'


# download data online
if not os.path.isfile(data_dir / file_name):

	pymrio.download_exiobase3(
	    storage_folder = data_dir,
	    system = system, 
	    years = base_year
	)


# import or build calibration data
if calib:

	# import exiobase data
	reference = pymrio.parse_exiobase3(
		data_dir / file_name
	)

	# isolate ghg emissions
	reference.ghg_emissions = Tools.extract_ghg_emissions(reference)

	# del useless extensions
	reference.remove_extension(['satellite', 'impacts'])

	# import agregation matrices
	agg_matrix = {
		key: pd.read_excel(
			data_dir / 'agg_matrix.xlsx',
			sheet_name = key + '_' + value
		) for (key, value) in agg_name.items()
	}
	agg_matrix['sector'].set_index(['category', 'sub_category', 'sector'], inplace = True)
	agg_matrix['region'].set_index(['Country name', 'Country code'], inplace = True)

	# apply regional and sectorial agregations
	reference.aggregate(
		region_agg = agg_matrix['region'].T.values,
		sector_agg = agg_matrix['sector'].T.values,
		region_names = agg_matrix['region'].columns.tolist(),
		sector_names = agg_matrix['sector'].columns.tolist()
	)

	# reset all to flows before saving
	reference = reference.reset_to_flows()
	reference.ghg_emissions.reset_to_flows()

	# save calibration data
	reference.save_all(
		data_dir / ('reference' + '_' + concat_settings)
	)

else:

	# import calibration data built with calib = True
	reference = pymrio.parse_exiobase3(
		data_dir / ('reference' + '_' + concat_settings)
	)


# CALCULATIONS

In [12]:
# calculate reference system
reference.calc_all()


# update extension calculations
reference.ghg_emissions_desag = Tools.recal_extensions_per_region(
	reference,
	'ghg_emissions'
)

# init counterfactual(s)
counterfactual = reference.copy()
counterfactual.remove_extension('ghg_emissions_desag')


sectors_list = list(reference.get_sectors())
reg_list = list(reference.get_regions())

# read param sets to shock reference system
## ToDo


# build conterfactual(s) using param sets
## ToDo


# calculate counterfactual(s) system
counterfactual.calc_all()
counterfactual.ghg_emissions_desag = Tools.recal_extensions_per_region(
	counterfactual,
	'ghg_emissions'
)

# VISUALIZE

In [6]:
dcba = reference.ghg_emissions_desag.D_cba
dpba = reference.ghg_emissions_desag.D_pba
dimp = reference.ghg_emissions_desag.D_imp
dexp = reference.ghg_emissions_desag.D_exp

# Desagrégation spatiale de l'enfer

In [7]:
reag_matrix = pd.read_excel(data_dir / 'agg_matrix_opti.xlsx', sheet_name = 'region_ref')
reag_matrix.columns[2:]

Index(['FR', 'United Kingdom', 'United States', 'Asia and Row Europe',
       'Chinafrica', 'Turkey and RoW America', 'Pacific and RoW Middle East',
       'Brazil, Mexico and South Africa', 'Switzerland and Norway',
       'RoW Asia and Pacific', 'EU'],
      dtype='object')

In [8]:
#create dic for region reaggregation :
dict_reag={}
for reg_agg in list(reag_matrix.columns[3:]):
    #print(reg_agg)
    list_reg_agg = []
    for i in reag_matrix.index:
        reg = reag_matrix.iloc[i].loc['Country name']
        #print(reg)
        if reag_matrix[reg_agg].iloc[i] == 1:
            list_reg_agg.append(reg)
    #print(list_reg_agg)
    dict_reag[reg_agg]=list_reg_agg
dict_reag['FR']=['FR']
dict_reag

{'United Kingdom': ['United Kingdom'],
 'United States': ['United States'],
 'Asia and Row Europe': ['Japan',
  'India',
  'Russia',
  'Indonesia',
  'RoW Europe'],
 'Chinafrica': ['China', 'RoW Africa'],
 'Turkey and RoW America': ['Canada', 'Turkey', 'RoW America'],
 'Pacific and RoW Middle East': ['South Korea',
  'Australia',
  'Taiwan',
  'RoW Middle East'],
 'Brazil, Mexico and South Africa': ['Brazil', 'Mexico', 'South Africa'],
 'Switzerland and Norway': ['Switzerland', 'Norway'],
 'RoW Asia and Pacific': ['RoW Asia and Pacific'],
 'EU': ['Austria',
  'Belgium',
  'Bulgaria',
  'Cyprus',
  'Czech Republic',
  'Germany',
  'Denmark',
  'Estonia',
  'Spain',
  'Finland',
  'Greece',
  'Croatia',
  'Hungary',
  'Ireland',
  'Italy',
  'Lithuania',
  'Luxembourg',
  'Latvia',
  'Malta',
  'Netherlands',
  'Poland',
  'Portugal',
  'Romania',
  'Sweden',
  'Slovenia',
  'Slovakia'],
 'FR': ['FR']}

In [9]:
#create dic for region reaggregation, reversed i.e. for each region, dic gives its new group :
dict_reag_new_group={}

for reg_agg in dict_reag :
    for reg in dict_reag[reg_agg]:
        dict_reag_new_group[reg] = reg_agg
#for sec in sectors_list:
#    dict_reag_new_group[sec] = sec
dict_reag_new_group

{'United Kingdom': 'United Kingdom',
 'United States': 'United States',
 'Japan': 'Asia and Row Europe',
 'India': 'Asia and Row Europe',
 'Russia': 'Asia and Row Europe',
 'Indonesia': 'Asia and Row Europe',
 'RoW Europe': 'Asia and Row Europe',
 'China': 'Chinafrica',
 'RoW Africa': 'Chinafrica',
 'Canada': 'Turkey and RoW America',
 'Turkey': 'Turkey and RoW America',
 'RoW America': 'Turkey and RoW America',
 'South Korea': 'Pacific and RoW Middle East',
 'Australia': 'Pacific and RoW Middle East',
 'Taiwan': 'Pacific and RoW Middle East',
 'RoW Middle East': 'Pacific and RoW Middle East',
 'Brazil': 'Brazil, Mexico and South Africa',
 'Mexico': 'Brazil, Mexico and South Africa',
 'South Africa': 'Brazil, Mexico and South Africa',
 'Switzerland': 'Switzerland and Norway',
 'Norway': 'Switzerland and Norway',
 'RoW Asia and Pacific': 'RoW Asia and Pacific',
 'Austria': 'EU',
 'Belgium': 'EU',
 'Bulgaria': 'EU',
 'Cyprus': 'EU',
 'Czech Republic': 'EU',
 'Germany': 'EU',
 'Denmark': 

In [13]:
M = reference.ghg_emissions_desag.M.copy()
ghg_list = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']


#create pd.MultiIndex for new_M
multi_reg = []
multi_sec = []
for reg in list(reag_matrix.columns[2:]) :
    for sec in sectors_list :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))
new_M = pd.DataFrame(np.zeros((len(ghg_list),len(sectors_list)*len(list(reag_matrix.columns[2:])))),
                              index =reference.ghg_emissions_desag.M.index,columns = new_col)


new_M['FR']=M['FR']
M
#print(M.columns)
    
for reg_agg in dict_reag:
    #print('\n', reg_agg)
    list_reg_agg = dict_reag[reg_agg]
    #print(list_reg_agg)
    for reg2 in list_reg_agg :
        #print(reg2)
        new_M[reg_agg] += M[reg2]/len(list_reg_agg)

new_M

region            FR                                                           \
sector   Agriculture Crude coal     Crude oil Natural gas Extractive industry   
stressor                                                                        
CO2         0.000590   0.003463  5.621522e-04    0.000903            0.000443   
CH4         0.001496   0.000118  4.279977e-03    0.000152            0.000066   
N2O         0.000875   0.000031  1.193598e-05    0.000011            0.000015   
SF6         0.000004   0.000002  1.310750e-06    0.000002            0.000026   
HFC         0.000134   0.000044  2.262622e-05    0.000014            0.001298   
PFC         0.000002   0.000001  8.968207e-07    0.000002            0.000009   

region                                                                      \
sector   Biomass_industry  Clothing Heavy_industry Construction Automobile   
stressor                                                                     
CO2              0.000429  0.000353       0.000729     0.000286   0.000377   
CH4              0.000373  0.000092       0.000110     0.000078   0.000080   
N2O              0.000196  0.000023       0.000019     0.000017   0.000014   
SF6              0.000002  0.000001       0.000003     0.000002   0.000003   
HFC              0.000056  0.000015       0.000070     0.000030   0.000018   
PFC              0.000002  0.000002       0.000004     0.000003   0.000003   

region    ...             EU                                                  \
sector    ... Heavy_industry Construction Automobile Oth transport equipment   
stressor  ...                                                                  
CO2       ...       0.000916     0.000346   0.000375                0.000552   
CH4       ...       0.000145     0.000075   0.000078                0.000068   
N2O       ...       0.000016     0.000010   0.000010                0.000010   
SF6       ...       0.000012     0.000004   0.000004                0.000003   
HFC       ...       0.000042     0.000019   0.000014                0.000012   
PFC       ...       0.000031     0.000007   0.000005                0.000004   

region                                                            \
sector   Machinery Electronics Fossil fuels Electricity and heat   
stressor                                                           
CO2       0.000358    0.000360     0.002934             0.003129   
CH4       0.000068    0.000061     0.001157             0.000263   
N2O       0.000009    0.000009     0.000012             0.000033   
SF6       0.000003    0.000002     0.000008             0.000006   
HFC       0.000012    0.000009     0.000030             0.000018   
PFC       0.000004    0.000003     0.000007             0.000003   

region                                 
sector   Transport services Composite  
stressor                               
CO2                0.000819  0.000207  
CH4                0.000086  0.000080  
N2O                0.000009  0.000008  
SF6                0.000002  0.000002  
HFC                0.000008  0.000011  
PFC                0.000002  0.000002  

[6 rows x 187 columns]

In [14]:
S = reference.ghg_emissions_desag.S.copy()
ghg_list = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']


#create pd.MultiIndex for new_S
multi_reg = []
multi_sec = []
for reg in list(reag_matrix.columns[2:]) :
    for sec in sectors_list :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))
new_S = pd.DataFrame(np.zeros((len(ghg_list),len(sectors_list)*len(list(reag_matrix.columns[2:])))),
                              index =reference.ghg_emissions_desag.S.index,columns = new_col)


new_S['FR']=S['FR']
S
#print(S.columns)
    
for reg_agg in dict_reag:
    #print('\n', reg_agg)
    list_reg_agg = dict_reag[reg_agg]
    #print(list_reg_agg)
    for reg2 in list_reg_agg :
        #print(reg2)
        new_S[reg_agg] += S[reg2]/len(list_reg_agg)

new_S

region              FR                                            \
sector     Agriculture    Crude coal     Crude oil   Natural gas   
stressor                                                           
CO2       2.752560e-04  3.210168e-03  3.354614e-04  5.992512e-04   
CH4       1.165031e-03  6.867609e-07  4.196768e-03  5.487065e-05   
N2O       7.196057e-04  2.072283e-05  5.189820e-06  2.943933e-07   
SF6       1.937657e-06  6.007967e-07  2.313296e-07  2.450581e-08   
HFC       1.034492e-04  3.217736e-05  1.401206e-05  1.257714e-06   
PFC       5.991865e-07  2.060316e-07  1.191699e-07  9.961699e-09   

region                                                                      \
sector   Extractive industry Biomass_industry      Clothing Heavy_industry   
stressor                                                                     
CO2             1.973631e-04     9.696668e-05  3.881976e-05   3.386476e-04   
CH4             5.066902e-07     3.193208e-06  5.591624e-07   1.760052e-06   
N2O             1.245002e-06     4.259090e-06  4.191614e-08   1.261228e-06   
SF6             2.420474e-05     3.464996e-07  3.176694e-08   2.747924e-07   
HFC             1.263246e-03     1.821899e-05  1.678283e-06   2.671380e-05   
PFC             7.261031e-06     1.074834e-07  9.150018e-09   1.117455e-06   

region                                ...             EU                \
sector    Construction    Automobile  ... Heavy_industry  Construction   
stressor                              ...                                
CO2       1.488762e-05  1.828081e-05  ...   3.886385e-04  5.911019e-05   
CH4       7.804559e-08  1.473360e-07  ...   9.974999e-06  5.584754e-06   
N2O       3.866893e-08  1.061538e-07  ...   9.873284e-07  4.449358e-07   
SF6       0.000000e+00  0.000000e+00  ...   4.467649e-06  2.430965e-07   
HFC       0.000000e+00  0.000000e+00  ...   1.268184e-05  1.148011e-06   
PFC       0.000000e+00  0.000000e+00  ...   1.632818e-05  4.029455e-07   

region                                                                      \
sector      Automobile Oth transport equipment     Machinery   Electronics   
stressor                                                                     
CO2       4.413741e-05            2.417216e-04  7.187185e-05  9.823409e-05   
CH4       1.603817e-06            1.160142e-06  2.949714e-06  1.952853e-06   
N2O       7.517405e-07            8.012262e-07  5.416436e-07  9.543006e-07   
SF6       5.925973e-08            6.786831e-08  1.881098e-07  5.722308e-08   
HFC       5.219439e-07            3.367309e-07  1.217698e-06  7.556944e-07   
PFC       1.946813e-08            3.908375e-08  3.964075e-07  4.465302e-07   

region                                                                        
sector    Fossil fuels Electricity and heat Transport services     Composite  
stressor                                                                      
CO2       2.406675e-03         2.437386e-03       5.675150e-04  4.462630e-05  
CH4       5.528467e-04         2.018583e-05       2.566451e-06  3.416558e-05  
N2O       9.469407e-07         2.214930e-05       2.980392e-06  5.510486e-07  
SF6       2.910415e-06         1.225997e-07       6.577529e-08  1.005559e-06  
HFC       1.464568e-05         8.296012e-07       4.950624e-07  4.540700e-06  
PFC       4.281670e-06         9.426002e-08       1.905644e-07  8.226175e-07  

[6 rows x 187 columns]

In [15]:
list_reg_reag_new=list(reag_matrix.columns[2:])

In [89]:
dcba = reference.ghg_emissions_desag.D_cba.copy()

ghg_list = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']

multi_reg = []
multi_sec = []
for reg in list_reg_reag_new :
    for sec in sectors_list :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

multi_reg2 = []
multi_ghg = []
for reg in list_reg_reag_new :
    for ghg in ghg_list :
        multi_reg2.append(reg)
        multi_ghg.append(ghg)
arrays2 = [multi_reg2, multi_ghg]
new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

new_dcba = pd.DataFrame(np.zeros((len(ghg_list)*len(list_reg_reag_new),
                                  len(sectors_list)*len(list_reg_reag_new))),
                              index =new_index,columns = new_col)

for reg_export in dict_reag :
    list_reg_agg_1 = dict_reag[reg_export]
    for reg_import in dict_reag :
        list_reg_agg_2 = dict_reag[reg_import]
        s1=pd.DataFrame(np.zeros_like(new_dcba.loc['FR','FR']),index=new_dcba.loc['FR','FR'].index, columns = new_dcba.loc['FR','FR'].columns)
        #print(s1.columns)
        for reg1 in list_reg_agg_1 :
            for reg2 in list_reg_agg_2 :
                #print(reg1,reg2)
                s1 += dcba.loc[reg1,reg2]
        
        for line in s1.index :
            for col in s1.columns :
                new_dcba.at[(reg_export,line),(reg_import,col)]=s1.loc[line,col]
        
new_dcba

region                   FR                                       \
sector          Agriculture Crude coal     Crude oil Natural gas   
region stressor                                                    
FR     CO2         4.796711   0.013821  3.149139e-05    2.894532   
       CH4        15.816831   0.000037  3.567591e-04    0.275431   
       N2O         9.715682   0.000093  5.603963e-07    0.007834   
       SF6         0.028011   0.000003  2.607044e-08    0.000477   
       HFC         1.487352   0.000147  1.475096e-06    0.024675   
...                     ...        ...           ...         ...   
EU     CH4         3.091531   0.000355  3.570174e-05    0.028653   
       N2O         1.669396   0.000009  3.575776e-07    0.003974   
       SF6         0.036347   0.000070  1.357071e-07    0.004541   
       HFC         0.165743   0.000017  5.493366e-07    0.007612   
       PFC         0.037457   0.000003  1.171452e-07    0.003592   

region                                                                         \
sector          Extractive industry Biomass_industry  Clothing Heavy_industry   
region stressor                                                                 
FR     CO2                 0.005818         9.866832  0.394570      13.261457   
       CH4                 0.000362         9.923741  0.075845       0.632317   
       N2O                 0.000127         6.010542  0.027877       0.196020   
       SF6                 0.000480         0.036355  0.000561       0.024654   
       HFC                 0.025029         1.894707  0.028297       1.641677   
...                             ...              ...       ...            ...   
EU     CH4                 0.000253         3.277704  0.257131       0.483021   
       N2O                 0.000066         1.750927  0.063645       0.107515   
       SF6                 0.000240         0.161043  0.018176       0.144685   
       HFC                 0.002472         0.397501  0.067395       0.288126   
       PFC                 0.000667         0.115331  0.035363       0.228889   

region                                   ...             EU               \
sector          Construction Automobile  ... Heavy_industry Construction   
region stressor                          ...                               
FR     CO2         14.029825   2.479608  ...       2.171746     1.680882   
       CH4          2.537475   0.319671  ...       0.190533     0.301424   
       N2O          0.968326   0.082451  ...       0.080943     0.142388   
       SF6          0.054173   0.003670  ...       0.004446     0.003608   
       HFC          2.874809   0.197846  ...       0.279117     0.206061   
...                      ...        ...  ...            ...          ...   
EU     CH4          0.871780   0.738130  ...       6.795545    22.206020   
       N2O          0.255239   0.142840  ...       1.141034     4.279811   
       SF6          0.133053   0.134929  ...       3.033871     3.721052   
       HFC          0.337786   0.252778  ...       5.002919     8.423468   
       PFC          0.224067   0.148901  ...       3.246368     3.064161   

region                                                                    \
sector          Automobile Oth transport equipment Machinery Electronics   
region stressor                                                            
FR     CO2        1.452006                0.551266  1.903225    0.424498   
       CH4        0.182520                0.077270  0.192376    0.046600   
       N2O        0.053332                0.018148  0.064001    0.016325   
       SF6        0.002249                0.000771  0.003134    0.000585   
       HFC        0.124646                0.038295  0.173902    0.030852   
...                    ...                     ...       ...         ...   
EU     CH4        2.374384                0.658824  4.628122    1.109084   
       N2O        0.421663                0.136212  0.813450    0.213190   
       SF6 

In [96]:
new_dcba['FR'].sum().sum()

579.7487732125793

## MCVE

In [88]:
list_reg_mcve = ['FR', 'United States', 'United Kingdom']
list_reg_mcve_new = ['FR','Other']
sectors_list_mcve = ['Agriculture','Composite']
dict_mcve = {'FR':['FR'],'Other' : ['United States', 'United Kingdom']}

ghg_list_mcve = ['CO2', 'CH4']

multi_reg = []
multi_sec = []
for reg in list_reg_mcve :
    for sec in sectors_list_mcve :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

multi_reg2 = []
multi_ghg = []
for reg in list_reg_mcve :
    for ghg in ghg_list_mcve :
        multi_reg2.append(reg)
        multi_ghg.append(ghg)
arrays2 = [multi_reg2, multi_ghg]
new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

dcba_mcve = pd.DataFrame(np.zeros((len(ghg_list_mcve)*len(list_reg_mcve),
                                   len(sectors_list_mcve)*len(list_reg_mcve))),
                              index =new_index,columns = new_col)

multi_reg = []
multi_sec = []
for reg in list_reg_mcve_new :
    for sec in sectors_list_mcve :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

multi_reg2 = []
multi_ghg = []
for reg in list_reg_mcve_new :
    for ghg in ghg_list_mcve :
        multi_reg2.append(reg)
        multi_ghg.append(ghg)
arrays2 = [multi_reg2, multi_ghg]
new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

dcba_mcve_new = pd.DataFrame(np.zeros((len(ghg_list_mcve)*len(list_reg_mcve_new),
                                   len(sectors_list_mcve)*len(list_reg_mcve_new))),
                              index =new_index,columns = new_col)
from random import randint
for col in dcba_mcve.columns:
    dcba_mcve[col]=dcba_mcve.apply(lambda x: randint(0,5), axis=1)


print(dcba_mcve)

for reg_export in dict_mcve :
    list_reg_agg_1 = dict_mcve[reg_export]
    for reg_import in dict_mcve :
        list_reg_agg_2 = dict_mcve[reg_import]
        s1=pd.DataFrame(np.zeros_like(dcba_mcve_new.loc['FR','FR']),index=dcba_mcve_new.loc['FR','FR'].index, columns = dcba_mcve_new.loc['FR','FR'].columns)
        for reg1 in list_reg_agg_1 :
            for reg2 in list_reg_agg_2 :
                #print(reg1,reg2)
                s1 += dcba_mcve.loc[reg1,reg2].copy()
        for line in s1.index :
            for col in s1.columns :
                dcba_mcve_new.at[(reg_export,line),(reg_import,col)]=s1.loc[line,col]
dcba_mcve_new


region                           FR           United States            \
sector                  Agriculture Composite   Agriculture Composite   
region         stressor                                                 
FR             CO2                1         0             1         3   
               CH4                5         2             4         1   
United States  CO2                3         0             0         5   
               CH4                0         4             1         3   
United Kingdom CO2                4         1             3         5   
               CH4                4         4             2         0   

region                  United Kingdom            
sector                     Agriculture Composite  
region         stressor                           
FR             CO2                   3         2  
               CH4                   4         0  
United States  CO2                   0         1  
               CH4                  

region                   FR                 Other          
sector          Agriculture Composite Agriculture Composite
region stressor                                            
FR     CO2              1.0       0.0         4.0       5.0
       CH4              5.0       2.0         8.0       1.0
Other  CO2              7.0       1.0         6.0      11.0
       CH4              4.0       8.0        11.0       5.0

## CVE

In [87]:
list_reg_mcve = reg_list #['FR', 'United States', 'United Kingdom']
list_reg_mcve_new = list_reg_reag_new #['FR','Other']
sectors_list_mcve = sectors_list #['Agriculture','Composite']
dict_mcve = dict_reag #{'FR':['FR'],'Other' : ['United States', 'United Kingdom']}

ghg_list_mcve = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']

multi_reg = []
multi_sec = []
for reg in list_reg_mcve :
    for sec in sectors_list_mcve :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

multi_reg2 = []
multi_ghg = []
for reg in list_reg_mcve :
    for ghg in ghg_list_mcve :
        multi_reg2.append(reg)
        multi_ghg.append(ghg)
arrays2 = [multi_reg2, multi_ghg]
new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

dcba_mcve = pd.DataFrame(np.zeros((len(ghg_list_mcve)*len(list_reg_mcve),
                                   len(sectors_list_mcve)*len(list_reg_mcve))),
                              index =new_index,columns = new_col)

multi_reg = []
multi_sec = []
for reg in list_reg_mcve_new :
    for sec in sectors_list_mcve :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

multi_reg2 = []
multi_ghg = []
for reg in list_reg_mcve_new :
    for ghg in ghg_list_mcve :
        multi_reg2.append(reg)
        multi_ghg.append(ghg)
arrays2 = [multi_reg2, multi_ghg]
new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

dcba_mcve_new = pd.DataFrame(np.zeros((len(ghg_list_mcve)*len(list_reg_mcve_new),
                                   len(sectors_list_mcve)*len(list_reg_mcve_new))),
                              index =new_index,columns = new_col)


from random import randint
for col in dcba_mcve.columns:
    dcba_mcve[col]=dcba_mcve.apply(lambda x: randint(0,5), axis=1)


for reg_export in dict_mcve :
    list_reg_agg_1 = dict_mcve[reg_export]
    for reg_import in dict_mcve :
        list_reg_agg_2 = dict_mcve[reg_import]
        s1=pd.DataFrame(np.zeros_like(dcba_mcve_new.loc['FR','FR']),index=dcba_mcve_new.loc['FR','FR'].index, columns = dcba_mcve_new.loc['FR','FR'].columns)
        for reg1 in list_reg_agg_1 :
            for reg2 in list_reg_agg_2 :
                s1 += dcba_mcve.loc[reg1,reg2]
        for line in s1.index :
            for col in s1.columns :
                dcba_mcve_new.at[(reg_export,line),(reg_import,col)]=s1.loc[line,col]
dcba_mcve_new

region                   FR                                   \
sector          Agriculture Crude coal Crude oil Natural gas   
region stressor                                                
FR     CO2              1.0        3.0       0.0         0.0   
       CH4              3.0        4.0       1.0         5.0   
       N2O              5.0        0.0       0.0         1.0   
       SF6              4.0        4.0       3.0         5.0   
       HFC              2.0        1.0       2.0         2.0   
...                     ...        ...       ...         ...   
EU     CH4             86.0       56.0      85.0        79.0   
       N2O             72.0       76.0      69.0        71.0   
       SF6             71.0       67.0      65.0        60.0   
       HFC             67.0       70.0      43.0        60.0   
       PFC             86.0       80.0      73.0        78.0   

region                                                                        \
sector          Extractive industry Biomass_industry Clothing Heavy_industry   
region stressor                                                                
FR     CO2                      0.0              2.0      4.0            3.0   
       CH4                      0.0              5.0      5.0            1.0   
       N2O                      0.0              0.0      5.0            1.0   
       SF6                      3.0              0.0      0.0            0.0   
       HFC                      5.0              1.0      4.0            2.0   
...                             ...              ...      ...            ...   
EU     CH4                     63.0             67.0     71.0           85.0   
       N2O                     77.0             69.0     56.0           62.0   
       SF6                     67.0             64.0     64.0           52.0   
       HFC                     75.0             72.0     65.0           79.0   
       PFC                     73.0             59.0     78.0           71.0   

region                                   ...             EU               \
sector          Construction Automobile  ... Heavy_industry Construction   
region stressor                          ...                               
FR     CO2               2.0        1.0  ...           64.0         59.0   
       CH4               2.0        4.0  ...           73.0         80.0   
       N2O               3.0        4.0  ...           72.0         74.0   
       SF6               4.0        4.0  ...           63.0         72.0   
       HFC               3.0        1.0  ...           60.0         66.0   
...                      ...        ...  ...            ...          ...   
EU     CH4              56.0       68.0  ...         1651.0       1668.0   
       N2O              55.0       88.0  ...         1659.0       1681.0   
       SF6              61.0       70.0  ...         1709.0       1669.0   
       HFC              66.0       48.0  ...         1776.0       1688.0   
       PFC              67.0       66.0  ...         1663.0       1656.0   

region                                                                    \
sector          Automobile Oth transport equipment Machinery Electronics   
region stressor                                                            
FR     CO2            61.0                    51.0      58.0        61.0   
       CH4            73.0                    57.0      63.0        77.0   
       N2O            54.0                    65.0      63.0        73.0   
       SF6            72.0                    61.0      85.0        60.0   
       HFC            70.0                    62.0      65.0        52.0   
...                    ...                     ...       ...         ...   
EU     CH4          1686.0                  1688.0    1664.0      1732.0   
       N2O          1663.0                  1652.0    1780.0      1582.0   
       SF6          1671.0                  1734.0    1722.0      1742.0   
     

In [66]:
df_slice.index
df_slice.columns
dcba_mcve_new.loc['FR','FR'].update(dcba_mcve.loc['FR','FR'])
print(all(dcba_mcve_new.loc['FR','FR']==dcba_mcve.loc['FR','FR']))
dcba_mcve_new.loc['FR','FR']

True


sector,Agriculture,Crude coal,Crude oil,Natural gas,Extractive industry,Biomass_industry,Clothing,Heavy_industry,Construction,Automobile,Oth transport equipment,Machinery,Electronics,Fossil fuels,Electricity and heat,Transport services,Composite
stressor,,,,,,,,,,,,,,,,,
CO2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CH4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N2O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SF6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HFC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PFC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
